# Day 5: Hydrothermal Venture
## Part One
You come across a field of <a href="https://en.wikipedia.org/wiki/Hydrothermal_vent" target="_blank">hydrothermal vents</a> on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in *lines*; the submarine helpfully produces a list of nearby <span title="Maybe they're Bresenham vents.">lines of vents</span> (your puzzle input) for you to review. For example:
```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```
<p>Each line of vents is given as a line segment in the format <code>x1,y1 -&gt; x2,y2</code> where <code>x1</code>,<code>y1</code> are the coordinates of one end the line segment and <code>x2</code>,<code>y2</code> are the coordinates of the other end. These line segments include the points at both ends. In other words:</p>
<ul>
<li>An entry like <code>1,1 -&gt; 1,3</code> covers points <code>1,1</code>, <code>1,2</code>, and <code>1,3</code>.</li>
<li>An entry like <code>9,7 -&gt; 7,7</code> covers points <code>9,7</code>, <code>8,7</code>, and <code>7,7</code>.</li>
</ul>
<p>For now, <em>only consider horizontal and vertical lines</em>: lines where either <code>x1 = x2</code> or <code>y1 = y2</code>.</p>
<p>So, the horizontal and vertical lines from the above list would produce the following diagram:</p>
<pre><code>.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
</code></pre>
<p>In this diagram, the top left corner is <code>0,0</code> and the bottom right corner is <code>9,9</code>. Each position is shown as <em>the number of lines which cover that point</em> or <code>.</code> if no line covers that point. The top-left pair of <code>1</code>s, for example, comes from <code>2,2 -&gt; 2,1</code>; the very bottom row is formed by the overlapping lines <code>0,9 -&gt; 5,9</code> and <code>0,9 -&gt; 2,9</code>.</p>
<p>To avoid the most dangerous areas, you need to determine <em>the number of points where at least two lines overlap</em>. In the above example, this is anywhere in the diagram with a <code>2</code> or larger - a total of <code><em>5</em></code> points.</p>
<p>Consider only horizontal and vertical lines. <em>At how many points do at least two lines overlap?</em></p>


### Solution

In [3]:
from dataclasses import dataclass


@dataclass(frozen=True)
class Point:
    x: int
    y: int

    @property
    def row(self) -> int:
        return self.y

    @property
    def col(self) -> int:
        return self.x


@dataclass(frozen=True)
class Line:
    point1: Point
    point2: Point

    def draw(self) -> list[Point]:
        if self.is_straight():
            return self._draw_straight()
        return self._draw_generic()

    def _draw_straight(self) -> list[Point]:
        if self._is_horizontal():
            return [Point(x, self.point1.y) for x in range(min(self.point1.x, self.point2.x), max(self.point1.x, self.point2.x) + 1)]
        return [Point(self.point1.x, y) for y in range(min(self.point1.y, self.point2.y), max(self.point1.y, self.point2.y) + 1)]

    def is_straight(self) -> bool:
        return self.point1.x == self.point2.x or self.point1.y == self.point2.y

    def _is_horizontal(self) -> bool:
        return self.point1.y == self.point2.y

    def _is_vertical(self) -> bool:
        return self.point1.x == self.point2.x
    
    def _draw_generic(self) -> list[Point]:
        dx = self.point2.x - self.point1.x
        dy = self.point2.y - self.point1.y
        diff = 2*dy - dx
        y = self.point1.y

        step = 1 if self.point1.x < self.point2.x else -1
        points: list[Point] = []
        for x in range(self.point1.x, self.point2.x + step, step):
            points.append(Point(x, y))
            if diff > 0:
                y += 1
                diff -= 2*dx
            diff += 2*dy
        return points

In [22]:
import numpy


class Grid:
    def __init__(self, x: int, y: int) -> None:
        self.grid = numpy.zeros(shape=(y, x), dtype=numpy.uint)

    def add_line(self, line: Line) -> None:
        points = line.draw()
        for point in points:
            self._add_point(point)

    def _add_point(self, point: Point) -> None:
        self.grid[point.row, point.col] += 1

    def dangerous_points(self) -> int:
        return (self.grid >= 2).sum()  # type: ignore

In [24]:
lines: list[Line] = []
max_x = max_y = 0
with open("input.txt", "r") as file:
    for line in file:
        (x1, y1), (x2, y2) = [map(int, p.split(","))
                              for p in line.strip().split(" -> ")]
        line = Line(
            Point(x1, y1),
            Point(x2, y2),
        )
        if not line.is_straight():
            continue
        lines.append(line)
        max_x = max(x1, x2, max_x)
        max_y = max(y1, y2, max_y)

In [25]:
grid = Grid(max_x + 1, max_y + 1)
for line in lines:
    grid.add_line(line)
print(grid.dangerous_points())

7269


## Part Two
<p>Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider <em>diagonal lines</em>.</p>
<p>Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:</p>
<ul>
<li>An entry like <code>1,1 -&gt; 3,3</code> covers points <code>1,1</code>, <code>2,2</code>, and <code>3,3</code>.</li>
<li>An entry like <code>9,7 -&gt; 7,9</code> covers points <code>9,7</code>, <code>8,8</code>, and <code>7,9</code>.</li>
</ul>
<p>Considering all lines from the above example would now produce the following diagram:</p>
<pre><code>1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
</code></pre>
<p>You still need to determine <em>the number of points where at least two lines overlap</em>. In the above example, this is still anywhere in the diagram with a <code>2</code> or larger - now a total of <code><em>12</em></code> points.</p>
<p>Consider all of the lines. <em>At how many points do at least two lines overlap?</em></p>

In [34]:
@dataclass(frozen=True)
class Line:
    point1: Point
    point2: Point

    def draw(self) -> list[Point]:
        if self.is_straight():
            return self._draw_straight()
        else:
            return self._draw_diagonal()

    def _draw_straight(self) -> list[Point]:
        if self._is_horizontal():
            return [Point(x, self.point1.y) for x in range(min(self.point1.x, self.point2.x), max(self.point1.x, self.point2.x) + 1)]
        else:
            return [Point(self.point1.x, y) for y in range(min(self.point1.y, self.point2.y), max(self.point1.y, self.point2.y) + 1)]

    def is_straight(self) -> bool:
        return self.point1.x == self.point2.x or self.point1.y == self.point2.y

    def _is_horizontal(self) -> bool:
        return self.point1.y == self.point2.y

    def _is_vertical(self) -> bool:
        return self.point1.x == self.point2.x
    
    def _draw_diagonal(self) -> list[Point]:
        step_x = 1 if self.point1.x < self.point2.x else -1
        range_x = range(self.point1.x, self.point2.x + step_x, step_x)
        step_y = 1 if self.point1.y < self.point2.y else -1
        range_y = range(self.point1.y, self.point2.y + step_y, step_y)
        return [Point(x, y) for x, y in zip(range_x, range_y)]

In [38]:
lines: list[Line] = []
max_x = max_y = 0
with open("input.txt", "r") as file:
    for line in file:
        (x1, y1), (x2, y2) = [map(int, p.split(","))
                              for p in line.strip().split(" -> ")]
        line = Line(
            Point(x1, y1),
            Point(x2, y2),
        )
        lines.append(line)
        max_x = max(x1, x2, max_x)
        max_y = max(y1, y2, max_y)

In [39]:
grid = Grid(max_x + 1, max_y + 1)
for line in lines:
    grid.add_line(line)
print(grid.dangerous_points())

21140
